## Taxi Fleet Manager Analysis: Revenue Optimization & Fleet Management

Focus: Day vs. Night operations, Empty miles, and Weather/Event impact

In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn jupyterlab folium plotly pyspark 

### Importing Libraries 

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import Window
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from pyspark.sql.functions import month
warnings.filterwarnings('ignore')

### Initialize Spark and Load data

In [3]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("TaxiDataFleetManager") \
    .getOrCreate()

# Path to processed parquet output
parquet_path = "taxi_processed.parquet"

# Read the processed parquet data
df = spark.read.parquet(parquet_path)

# Show schema and a sample of the data
print("Schema:")
df.printSchema()

print("\nSample data:")
df.show(5, truncate=False)

# Optionally, count records
print(f"\nTotal records: {df.count():,}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/24 21:59:52 WARN Utils: Your hostname, Kaweewat.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
25/11/24 21:59:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/24 21:59:52 WARN Utils: Your hostname, Kaweewat.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
25/11/24 21:59:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.

Schema:
root
 |-- vehicle_id: string (nullable = true)
 |-- date_only: date (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- heading: integer (nullable = true)
 |-- for_hire_light: integer (nullable = true)
 |-- engine_acc: integer (nullable = true)
 |-- gpsvalid: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- day_name: string (nullable = true)
 |-- is_weekend: integer (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- day_night_shift: string (nullable = true)
 |-- is_rush_hour: integer (nullable = true)
 |-- is_hired: integer (nullable = true)
 |-- is_searching_fare: integer (nullable = true)
 |-- is_idle: integer (nullable = true)
 |-- is_moving: integer (nullable = true)
 |-- is_stationary: integer (nullable = true)
 |-- speed_category: string (nullable = true)
 |-- distance_from_b

25/11/24 21:59:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------------+----------+-------------------+--------+---------+-----+-------+--------------+----------+--------+----+-----------+--------+----------+-----------+---------------+------------+--------+-----------------+-------+---------+-------------+--------------+------------------------+------------------------------+-----------------------------+---------------------------+-----------------------------+-----------------------------+---------+--------+--------+-----------------------------+----------------+------------------+------------+--------------------------+--------------------------+---------------------------+------------------------+-------------------+---------------+-------------------------+--------------------+-----------------+--------------------+--------------------+----------+--------+------------------+-------------+------------+-----------------+---------------------+----------------+------------------+---------------+-----------------------+-------

In [4]:
# Each block should define its own data_copy from df for independence
data_copy = df

In [5]:
# ============================================================================
# 1A: DAY vs NIGHT REVENUE ANALYSIS (WITH THAILAND TAXI METER ESTIMATION)
# ============================================================================
print("\n" + "="*80)
print("[1] REVENUE PREDICTION & VEHICLE OPTIMIZATION (THAILAND METER)")
print("="*80)

# ---------------------------------------------------------------------------
# Thailand Taxi Fare UDF
# ---------------------------------------------------------------------------
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

def calc_fare(distance_km, stopped_min):
    if distance_km is None:
        distance_km = 0
    if stopped_min is None:
        stopped_min = 0

    # Base fare
    fare = 35.0

    # Distance segments
    remaining = max(distance_km, 0)

    # 1–10 km
    if remaining > 1:
        segment = min(remaining - 1, 9)
        fare += segment * 6.5

    # 10–20 km
    if remaining > 10:
        segment = min(remaining - 10, 10)
        fare += segment * 7.0

    # 20–40 km
    if remaining > 20:
        segment = min(remaining - 20, 20)
        fare += segment * 8.0

    # 40–60 km
    if remaining > 40:
        segment = min(remaining - 40, 20)
        fare += segment * 8.5

    # 60–80 km
    if remaining > 60:
        segment = min(remaining - 60, 20)
        fare += segment * 9.0

    # >80 km
    if remaining > 80:
        segment = remaining - 80
        fare += segment * 10.5

    # Add traffic waiting fare
    fare += stopped_min * 3.0  # 3 baht per min below 6 km/h

    return float(fare)

fare_udf = udf(calc_fare, FloatType())

# ---------------------------------------------------------------------------
# ESTIMATE STOPPED TIME (speed < 6 km/h)
# ---------------------------------------------------------------------------
data_copy = data_copy.withColumn(
    "stopped_minutes",
    F.when(data_copy.is_hired == 1, 
           F.when(data_copy.speed < 6, 1).otherwise(0)
    ).otherwise(0)
)

# ---------------------------------------------------------------------------
# CALCULATE TRIP FARE (When hired)
# ---------------------------------------------------------------------------
data_copy = data_copy.withColumn(
    "estimated_fare",
    F.when(data_copy.is_hired == 1,
           fare_udf(data_copy.distance_traveled_km, data_copy.stopped_minutes)
    ).otherwise(0)
)

# ---------------------------------------------------------------------------
# SHIFT-LEVEL REVENUE METRICS
# ---------------------------------------------------------------------------
shift_revenue = data_copy.groupBy('day_night_shift').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_pings'),
    F.sum('estimated_fare').alias('total_revenue_baht'),
    F.countDistinct('vehicle_id').alias('active_vehicles'),
    F.avg('speed').alias('avg_speed_kmh'),
    F.sum('distance_traveled_km').alias('total_distance_km')
).toPandas()

print("\n📊 SHIFT PERFORMANCE SUMMARY:")
print(shift_revenue.to_string(index=False))

for idx, row in shift_revenue.iterrows():
    shift = row['day_night_shift']
    total = row['total_pings']
    hired = row['hired_pings']
    revenue = row['total_revenue_baht']
    util_rate = (hired / total * 100) if total > 0 else 0
    
    print(f"\n🔹 {shift} Shift Analysis:")
    print(f"   • Utilization Rate: {util_rate:.2f}%")
    print(f"   • Total Revenue: {revenue:,.2f} Baht")
    print(f"   • Total Pings: {int(total):,}")
    print(f"   • Hired Pings: {int(hired):,}")
    print(f"   • Active Vehicles: {int(row['active_vehicles'])}")
    print(f"   • Average Speed: {row['avg_speed_kmh']:.2f} km/h")
    print(f"   • Total Distance: {row['total_distance_km']:,.0f} km")

# ============================================================================
# HOURLY AVERAGE REVENUE PER VEHICLE ACROSS THE DAY
# ============================================================================
print("\n\n📊 HOURLY AVERAGE REVENUE PER VEHICLE (ALL SHIFTS COMBINED):")

# Aggregate by hour
hourly_avg = data_copy.groupBy('hour').agg(
    F.sum('estimated_fare').alias('total_revenue'),
    F.countDistinct('vehicle_id').alias('active_vehicles'),
    F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_pings')
).toPandas().sort_values('hour')

# Compute average revenue per vehicle
hourly_avg['avg_revenue_per_vehicle'] = hourly_avg['total_revenue'] / hourly_avg['active_vehicles']

# Format for readability
hourly_avg['total_revenue'] = hourly_avg['total_revenue'].apply(lambda x: f"{int(x):,}")
hourly_avg['avg_revenue_per_vehicle'] = hourly_avg['avg_revenue_per_vehicle'].apply(lambda x: f"{int(x):,}")

print(hourly_avg.to_string(index=False))



[1] REVENUE PREDICTION & VEHICLE OPTIMIZATION (THAILAND METER)



📊 SHIFT PERFORMANCE SUMMARY:
day_night_shift  total_pings  hired_pings  total_revenue_baht  active_vehicles  avg_speed_kmh  total_distance_km
      day_shift      3455684       707476        3.800473e+07             3246      11.190372       5.939412e+06
    night_shift      4051516      1225114        6.373050e+07             3254      14.964312       9.694708e+06

🔹 day_shift Shift Analysis:
   • Utilization Rate: 20.47%
   • Total Revenue: 38,004,733.56 Baht
   • Total Pings: 3,455,684
   • Hired Pings: 707,476
   • Active Vehicles: 3246
   • Average Speed: 11.19 km/h
   • Total Distance: 5,939,412 km

🔹 night_shift Shift Analysis:
   • Utilization Rate: 30.24%
   • Total Revenue: 63,730,501.77 Baht
   • Total Pings: 4,051,516
   • Hired Pings: 1,225,114
   • Active Vehicles: 3254
   • Average Speed: 14.96 km/h
   • Total Distance: 9,694,708 km


📊 HOURLY AVERAGE REVENUE PER VEHICLE (ALL SHIFTS COMBINED):


 hour total_revenue  active_vehicles  hired_pings avg_revenue_per_vehicle
    0     5,366,336             3207       106555                   1,673
    1     5,118,508             3191       101235                   1,604
    2     4,980,423             3170        96338                   1,571
    3     4,702,145             3168        88253                   1,484
    4     4,324,468             3145        79389                   1,375
    5     3,682,005             3141        66895                   1,172
    6     3,039,079             3142        55249                     967
    7     2,412,157             3135        43901                     769
    8     1,943,279             3132        34645                     620
    9     1,659,515             3137        30004                     529
   10     1,635,517             3130        29401                     522
   11     1,781,589             3141        31605                     567
   12     2,557,196             3157  

In [6]:
# Independent data snapshot for this block
data_copy = df

In [7]:
# ============================================================================
# 1B: EMPTY MILES & OPERATIONAL EFFICIENCY
# ============================================================================
print("\n\n" + "="*80)
print("[2] EMPTY MILES ANALYSIS - OPERATIONAL EFFICIENCY")
print("="*80)

# More robust empty analysis
efficiency_by_shift_hour = data_copy.groupBy('day_night_shift', 'hour').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when(data_copy.is_idle == 1, 1).otherwise(0)).alias('idle_pings'),
    F.sum(F.when(data_copy.is_searching_fare == 1, 1).otherwise(0)).alias('searching_pings'),
    F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_pings'),
    F.sum(F.when(data_copy.is_moving == 1, 1).otherwise(0)).alias('moving_pings'),
    F.sum('distance_traveled_km').alias('distance_km'),
    F.countDistinct('vehicle_id').alias('vehicles')
).toPandas()

# Calculate efficiency metrics
efficiency_by_shift_hour['empty_rate'] = (
    efficiency_by_shift_hour['idle_pings'] + efficiency_by_shift_hour['searching_pings']
 ) / efficiency_by_shift_hour['total_pings']

efficiency_by_shift_hour['utilization_rate'] = efficiency_by_shift_hour['hired_pings'] / efficiency_by_shift_hour['total_pings']

efficiency_by_shift_hour['avg_distance_per_ping'] = efficiency_by_shift_hour['distance_km'] / efficiency_by_shift_hour['total_pings']

print("\n⚙️  SHIFT EFFICIENCY METRICS:")
for shift in ['Day', 'Night']:
    shift_data = efficiency_by_shift_hour[efficiency_by_shift_hour['day_night_shift'] == shift]

    if shift_data.empty:
        print(f"\n{shift} Shift:")
        print("   • No data available for this shift.")
        continue

    avg_empty = shift_data['empty_rate'].mean() * 100
    avg_util = shift_data['utilization_rate'].mean() * 100

    peak_empty_hour = shift_data.loc[shift_data['empty_rate'].idxmax(), 'hour']
    peak_empty_rate = shift_data['empty_rate'].max() * 100

    peak_util_hour = shift_data.loc[shift_data['utilization_rate'].idxmax(), 'hour']
    peak_util_rate = shift_data['utilization_rate'].max() * 100

    print(f"\n{shift} Shift:")
    print(f"   • Average Empty Rate: {avg_empty:.2f}%")
    print(f"   • Average Utilization Rate: {avg_util:.2f}%")
    print(f"   • Peak Empty Hour: {int(peak_empty_hour):02d}:00 ({peak_empty_rate:.2f}%)")
    print(f"   • Peak Utilization Hour: {int(peak_util_hour):02d}:00 ({peak_util_rate:.2f}%)")

# High-waste periods
waste_threshold = efficiency_by_shift_hour['empty_rate'].quantile(0.75)
high_waste = efficiency_by_shift_hour[efficiency_by_shift_hour['empty_rate'] > waste_threshold]

print(f"\n⚠️  HIGH-WASTE PERIODS (Top 25% - Empty Rate > {waste_threshold*100:.2f}%): {len(high_waste)} periods identified")
print("\n   Hours to optimize:")
for idx, row in high_waste.sort_values('empty_rate', ascending=False).head(5).iterrows():
    print(f"      • {row['day_night_shift']} {int(row['hour']):02d}:00 - Empty Rate: {row['empty_rate']*100:.2f}%")




[2] EMPTY MILES ANALYSIS - OPERATIONAL EFFICIENCY



⚙️  SHIFT EFFICIENCY METRICS:

Day Shift:
   • No data available for this shift.

Night Shift:
   • No data available for this shift.

⚠️  HIGH-WASTE PERIODS (Top 25% - Empty Rate > 81.46%): 6 periods identified

   Hours to optimize:
      • day_shift 10:00 - Empty Rate: 88.32%
      • day_shift 09:00 - Empty Rate: 88.01%
      • day_shift 11:00 - Empty Rate: 87.51%
      • day_shift 08:00 - Empty Rate: 86.30%
      • day_shift 07:00 - Empty Rate: 83.78%


In [8]:
# Independent data snapshot for this block
data_copy = df

In [9]:
# ============================================================================
# 1C: EMPTY DISTANCE ANALYSIS
# ============================================================================
print("\n\n📍 EMPTY MILES BREAKDOWN:")

# Analyze when vehicles are moving but not hired (searching/idle while moving)
movement_analysis = data_copy.groupBy('day_night_shift').agg(
    F.sum(F.when((data_copy.is_searching_fare == 1) & (data_copy.is_moving == 1), data_copy.distance_traveled_km).otherwise(0)).alias('searching_distance_km'),
    F.sum(F.when((data_copy.is_hired == 1) & (data_copy.is_moving == 1), data_copy.distance_traveled_km).otherwise(0)).alias('hired_distance_km'),
    F.sum(F.when((data_copy.is_idle == 1), data_copy.distance_traveled_km).otherwise(0)).alias('idle_distance_km'),
).toPandas()

for idx, row in movement_analysis.iterrows():
    shift = row.index if isinstance(row.index, str) else 'Shift'
    total_dist = row['searching_distance_km'] + row['hired_distance_km'] + row['idle_distance_km']
    
    search_pct = (row['searching_distance_km'] / total_dist * 100) if total_dist > 0 else 0
    hired_pct = (row['hired_distance_km'] / total_dist * 100) if total_dist > 0 else 0
    
print("\nDistance Distribution by Status:")
print(movement_analysis.to_string(index=False))



📍 EMPTY MILES BREAKDOWN:

Distance Distribution by Status:
day_night_shift  searching_distance_km  hired_distance_km  idle_distance_km
      day_shift           2.462119e+06       1.789421e+06     469183.394863
    night_shift           4.047659e+06       2.794279e+06     669545.227493

Distance Distribution by Status:
day_night_shift  searching_distance_km  hired_distance_km  idle_distance_km
      day_shift           2.462119e+06       1.789421e+06     469183.394863
    night_shift           4.047659e+06       2.794279e+06     669545.227493


In [10]:
# Independent data snapshot for this block
data_copy = df

In [11]:
# ============================================================================
# 1D: TOURISM & LOCATION-BASED DEMAND PATTERNS
# ============================================================================
print("\n\n" + "="*80)
print("[3] TOURISM & LOCATION-BASED DEMAND PATTERNS")
print("="*80)

# Tourist hub analysis
tourist_analysis = data_copy.groupBy('is_tourist_time', 'day_night_shift').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_pings'),
    F.countDistinct('vehicle_id').alias('active_vehicles'),
    F.avg('speed').alias('avg_speed')
).toPandas()

print("\n🎯 TOURIST ACTIVITY IMPACT:")
for idx, row in tourist_analysis.iterrows():
    time_label = "Tourist Hours" if row['is_tourist_time'] == 1 else "Off-Peak Hours"
    util = (row['hired_pings'] / row['total_pings'] * 100) if row['total_pings'] > 0 else 0
    
    print(f"\n{time_label} ({row['day_night_shift']} Shift):")
    print(f"   • Utilization Rate: {util:.2f}%")
    print(f"   • Active Vehicles: {int(row['active_vehicles'])}")
    print(f"   • Average Speed: {row['avg_speed']:.2f} km/h")

# Airport vicinity analysis
airport_analysis = data_copy.groupBy('is_airport_fare_area', 'day_night_shift').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_pings'),
    F.avg('speed').alias('avg_speed'),
    F.countDistinct('vehicle_id').alias('active_vehicles'),
    F.sum('distance_traveled_km').alias('distance_km')
).toPandas()

print("\n✈️  AIRPORT ZONE PERFORMANCE:")
airport_data = airport_analysis[airport_analysis['is_airport_fare_area'] == 1]
for idx, row in airport_data.iterrows():
    util = (row['hired_pings'] / row['total_pings']) * 100 if row['total_pings'] > 0 else 0
    print(f"\n{row['day_night_shift']} Shift:")
    print(f"   • Utilization: {util:.2f}%")
    print(f"   • Active Vehicles: {int(row['active_vehicles'])}")
    print(f"   • Average Speed: {row['avg_speed']:.2f} km/h")
    print(f"   • Total Distance: {row['distance_km']:,.0f} km")

# Area type performance
print("\n🗺️  PERFORMANCE BY AREA TYPE:")
area_performance = data_copy.groupBy('area_type').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_pings'),
    F.countDistinct('vehicle_id').alias('vehicles'),
    F.avg('speed').alias('avg_speed')
).toPandas().sort_values('hired_pings', ascending=False)

for idx, row in area_performance.iterrows():
    util = (row['hired_pings'] / row['total_pings'] * 100) if row['total_pings'] > 0 else 0
    if pd.notna(row['area_type']):
        print(f"\n{row['area_type']}:")
        print(f"   • Utilization: {util:.2f}% | Vehicles: {int(row['vehicles'])} | Avg Speed: {row['avg_speed']:.2f} km/h")




[3] TOURISM & LOCATION-BASED DEMAND PATTERNS



🎯 TOURIST ACTIVITY IMPACT:

Off-Peak Hours (day_shift Shift):
   • Utilization Rate: 19.31%
   • Active Vehicles: 3219
   • Average Speed: 9.90 km/h

Tourist Hours (night_shift Shift):
   • Utilization Rate: 29.47%
   • Active Vehicles: 3202
   • Average Speed: 15.32 km/h

Tourist Hours (day_shift Shift):
   • Utilization Rate: 20.95%
   • Active Vehicles: 3239
   • Average Speed: 11.71 km/h

Off-Peak Hours (night_shift Shift):
   • Utilization Rate: 30.54%
   • Active Vehicles: 3245
   • Average Speed: 14.83 km/h



✈️  AIRPORT ZONE PERFORMANCE:

night_shift Shift:
   • Utilization: 100.00%
   • Active Vehicles: 1669
   • Average Speed: 17.04 km/h
   • Total Distance: 51,508 km

day_shift Shift:
   • Utilization: 100.00%
   • Active Vehicles: 1482
   • Average Speed: 20.28 km/h
   • Total Distance: 40,803 km

🗺️  PERFORMANCE BY AREA TYPE:



bangkok_inner_ring:
   • Utilization: 26.38% | Vehicles: 3156 | Avg Speed: 13.32 km/h

bangkok_outer_belt:
   • Utilization: 22.98% | Vehicles: 3191 | Avg Speed: 12.71 km/h

bangkok_center_core:
   • Utilization: 36.39% | Vehicles: 2907 | Avg Speed: 13.94 km/h

bmr_suburban_area:
   • Utilization: 23.45% | Vehicles: 3038 | Avg Speed: 14.63 km/h


In [12]:
# Independent data snapshot for this block
data_copy = df

In [13]:
# ============================================================================
# 1E: VEHICLE PERFORMANCE RANKING
# ============================================================================
print("\n\n" + "="*80)
print("[4] VEHICLE PERFORMANCE RANKING")
print("="*80)

vehicle_performance = data_copy.groupBy('vehicle_id').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_pings'),
    F.sum(F.when(data_copy.is_searching_fare == 1, 1).otherwise(0)).alias('searching_pings'),
    F.sum('distance_traveled_km').alias('total_distance'),
    F.max('daily_trip_starts').alias('trip_count'),
    F.avg('speed').alias('avg_speed')
).withColumn(
    'efficiency_score', F.col('hired_pings') / F.col('total_pings')
).orderBy(F.col('efficiency_score').desc()).limit(15).toPandas()

print("\n🏆 TOP 15 MOST EFFICIENT VEHICLES:")
print(vehicle_performance[['vehicle_id', 'efficiency_score', 'hired_pings', 'total_pings', 'total_distance']].to_string(index=False))



[4] VEHICLE PERFORMANCE RANKING



🏆 TOP 15 MOST EFFICIENT VEHICLES:
                 vehicle_id  efficiency_score  hired_pings  total_pings  total_distance
0CBePn0wxdYO/wQYsEmLuI5TzNY               1.0          216          216      963.728441
MqerR3AuusKyLVPE/AvZxG4IEJk               1.0          282          282     1913.333011
qOmIpRnN7UZPSEtuKK07ir/jZ1w               1.0           43           43      425.471571
Ad8mJMyRvDDE7A7l4usM6fqWHxE               1.0          464          464     3556.527708
owSjcBz32Mbzs4bjBCI3dXkhcJ4               1.0            1            1        0.010278
UobOJ1VcbzgQeryuhS/pWotU0FM               1.0           26           26      244.638740
GEy44fHfDq9x2g3C7NLS6uKJ+OA               1.0         4830         4830     3818.003478
TMMrCwVW/qfpWDNWJlBeg3sIAPA               1.0          187          187      813.704318
tuTiA33nFWMVJZqmgNt0UW8gwso               1.0         3068         3068     2676.416138
rOXSUeaLaa5mpN8eibg3aJ8blog               1.0           76           76      664.0651

In [14]:
# Independent data snapshot for this block
data_copy = df

In [15]:
# ============================================================================
# 1F EMPTY PASSENGER RATE BY AREA (TOURIST VS NON-TOURIST)
# ============================================================================
area_empty = data_copy.groupBy('area_type', 'is_tourist_time').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when((data_copy.is_idle == 1) | (data_copy.is_searching_fare == 1), 1).otherwise(0)).alias('empty_pings')
).withColumn(
    'empty_rate', F.col('empty_pings') / F.col('total_pings')
).toPandas()

area_empty['tourist_label'] = area_empty['is_tourist_time'].map({1: 'Tourist', 0: 'Non-Tourist'})
area_empty = area_empty.sort_values('empty_rate', ascending=False)

print("🚕 Empty Passenger Rate by Area (Tourist vs Non-Tourist):")
for idx, row in area_empty.iterrows():
    print(f"{row['area_type']} ({row['tourist_label']}): {row['empty_rate']*100:.2f}% empty rate ({int(row['empty_pings'])}/{int(row['total_pings'])})")

🚕 Empty Passenger Rate by Area (Tourist vs Non-Tourist):
bangkok_outer_belt (Tourist): 78.58% empty rate (1212251/1542611)
bmr_suburban_area (Tourist): 77.68% empty rate (221934/285706)
bangkok_inner_ring (Tourist): 76.05% empty rate (1099553/1445785)
bangkok_outer_belt (Non-Tourist): 75.50% empty rate (1207444/1599179)
bmr_suburban_area (Non-Tourist): 75.43% empty rate (218847/290117)
bangkok_inner_ring (Non-Tourist): 71.40% empty rate (1133530/1587610)
bangkok_center_core (Tourist): 66.05% empty rate (220550/333929)
bangkok_center_core (Non-Tourist): 61.69% empty rate (260501/422263)


### Visualization

In [16]:
data_copy = df

In [17]:
# Ensure numeric types for plotting (handle both string and numeric cases)
if hourly_avg['avg_revenue_per_vehicle'].dtype == 'O':
    hourly_avg['avg_revenue_per_vehicle'] = pd.to_numeric(hourly_avg['avg_revenue_per_vehicle'].astype(str).str.replace(',', ''), errors='coerce')
if hourly_avg['total_revenue'].dtype == 'O':
    hourly_avg['total_revenue'] = pd.to_numeric(hourly_avg['total_revenue'].astype(str).str.replace(',', ''), errors='coerce')

# Create interactive line chart
fig = px.line(
    hourly_avg,
    x='hour',
    y='avg_revenue_per_vehicle',
    title='📊 Hourly Average Revenue per Vehicle',
    labels={
        'hour': 'Hour of Day',
        'avg_revenue_per_vehicle': 'Average Revenue per Vehicle (Baht)'
    },
    markers=True
)

# Optional: add total revenue as secondary y-axis
fig.add_bar(
    x=hourly_avg['hour'],
    y=hourly_avg['total_revenue'],
    name='Total Revenue',
    opacity=0.3,
    yaxis='y2'
)

# Add secondary y-axis layout
fig.update_layout(
    yaxis2=dict(
        title='Total Revenue (Baht)',
        overlaying='y',
        side='right'
    ),
    xaxis=dict(dtick=1),
    template='plotly_white'
)

# Save to interactive HTML
fig.write_html("01_hourly_avg_revenue.html")

print("✅ Interactive HTML chart saved as 'hourly_avg_revenue.html'")


✅ Interactive HTML chart saved as 'hourly_avg_revenue.html'


In [18]:
# Chart 2: Operational Efficiency Heatmap
efficiency_pivot = efficiency_by_shift_hour.pivot_table(
    values='empty_rate', index='hour', columns='day_night_shift', aggfunc='first'
) * 100

fig2 = go.Figure(data=go.Heatmap(
    z=efficiency_pivot.values,
    x=efficiency_pivot.columns,
    y=efficiency_pivot.index,
    colorscale='RdYlGn_r',
    text=np.round(efficiency_pivot.values, 1),
    texttemplate='%{text:.1f}%',
    colorbar=dict(title="Empty Rate %", thickness=20)
))

fig2.update_layout(
    title='Operational Empty Rate by Hour (Lower % = Better Efficiency)',
    xaxis_title='Shift',
    yaxis_title='Hour of Day',
    height=600,
    font=dict(size=12)
)
fig2.write_html('02_efficiency_heatmap.html')

In [19]:
# Chart 3: Utilization vs Empty Rate Scatter
fig3 = px.scatter(efficiency_by_shift_hour, x='utilization_rate', y='empty_rate',
                  color='day_night_shift', size='vehicles', hover_data=['hour'],
                  title='Utilization vs Empty Rate Analysis',
                  labels={'utilization_rate': 'Utilization Rate', 'empty_rate': 'Empty Rate', 'vehicles': 'Active Vehicles'},
                  color_discrete_map={'Day': '#1f77b4', 'Night': '#ff7f0e'})
fig3.update_traces(marker=dict(line=dict(width=1, color='white')))
fig3.update_layout(height=600, hovermode='closest', template='plotly')
fig3.write_html('03_utilization_scatter.html')

In [20]:
# Chart 4: Tourist Impact Analysis
# Ensure 'is_tourist_time' column exists before any groupBy/groupby using it
if hasattr(data_copy, 'withColumn'):
    # Spark DataFrame
    if 'is_tourist_time' not in data_copy.columns:
        data_copy = data_copy.withColumn(
            'is_tourist_time',
            F.when((data_copy['hour'] >= 10) & (data_copy['hour'] <= 22), 1).otherwise(0)
        )
else:
    # Pandas DataFrame
    if 'is_tourist_time' not in data_copy.columns:
        data_copy['is_tourist_time'] = ((data_copy['hour'] >= 10) & (data_copy['hour'] <= 22)).astype(int)

if hasattr(data_copy, 'groupBy'):
    tourist_hourly = data_copy.groupBy('is_tourist_time', 'day_night_shift', 'hour').agg(
        F.sum(F.when(data_copy.is_hired == 1, 1).otherwise(0)).alias('hired_count')
    ).toPandas()
else:
    tourist_hourly = data_copy.groupby(['is_tourist_time', 'day_night_shift', 'hour']).agg({
        'is_hired': 'sum'
    }).reset_index().rename(columns={'is_hired': 'hired_count'})

fig4 = px.line(tourist_hourly, x='hour', y='hired_count', 
               color='day_night_shift', facet_col='is_tourist_time',
               markers=True, 
               title='Demand Patterns: Tourist Hours vs Off-Peak',
               labels={'is_tourist_time': 'Tourist Activity', 'hour': 'Hour', 'hired_count': 'Hired Pings'},
               facet_col_wrap=2)
fig4.for_each_annotation(lambda a: a.update(text=a.text.replace("is_tourist_time = ", "")))
fig4.write_html('04_tourist_impact.html')

### Map Visualization

In [21]:
import folium
from folium.plugins import HeatMap
import pandas as pd

# ============================================================================
# EMPTY PASSENGER RATE BY LAT/LON (TOURIST VS NON-TOURIST)
# ============================================================================
# Aggregate by grid or exact lat/lon
empty_geo = df.groupBy('lat_grid', 'lon_grid', 'is_tourist_time').agg(
    F.count('*').alias('total_pings'),
    F.sum(F.when((df.is_idle == 1) | (df.is_searching_fare == 1), 1).otherwise(0)).alias('empty_pings')
).withColumn(
    'empty_rate', F.col('empty_pings') / F.col('total_pings')
).toPandas()

# Map labels
empty_geo['tourist_label'] = empty_geo['is_tourist_time'].map({1: 'Tourist', 0: 'Non-Tourist'})

# ============================================================================
# CREATE INTERACTIVE MAP
# ============================================================================
# Center map at mean coordinates
map_center = [empty_geo['lat_grid'].mean(), empty_geo['lon_grid'].mean()]
m = folium.Map(location=map_center, zoom_start=12)

# Add circles for each grid point
for _, row in empty_geo.iterrows():
    folium.Circle(
        location=[row['lat_grid'], row['lon_grid']],
        radius=50,  # size of circle in meters
        color='red' if row['tourist_label']=='Tourist' else 'blue',
        fill=True,
        fill_opacity=row['empty_rate'],  # opacity represents empty rate
        popup=(
            f"Empty Rate: {row['empty_rate']*100:.2f}%\n"
            f"Total Pings: {row['total_pings']}\n"
            f"Empty Pings: {row['empty_pings']}\n"
            f"Type: {row['tourist_label']}"
        )
    ).add_to(m)

# Optionally, add a heatmap overlay
heat_data = [[row['lat_grid'], row['lon_grid'], row['empty_rate']] for _, row in empty_geo.iterrows()]
HeatMap(heat_data, radius=15, blur=10, max_zoom=12).add_to(m)

# Save map
m.save('empty_passenger_map.html')
print("✅ Interactive map saved as empty_passenger_map.html")


✅ Interactive map saved as empty_passenger_map.html


Cluster/Marker Maps

In [22]:
# Additional Visualization Map: Top Empty Rate Hotspots (Tourist vs Non-Tourist)
import folium
from folium.plugins import MarkerCluster

# Use the empty_geo DataFrame from previous map cell
# Find top N hotspots with highest empty rate for each type
N = 10
hotspots = empty_geo.sort_values('empty_rate', ascending=False).groupby('tourist_label').head(N)

# Create a new map centered at the mean coordinates
hotspot_map = folium.Map(location=map_center, zoom_start=12)

# Add marker clusters for each type
for label, color in zip(['Tourist', 'Non-Tourist'], ['red', 'blue']):
    cluster = MarkerCluster(name=f"{label} Hotspots").add_to(hotspot_map)
    for _, row in hotspots[hotspots['tourist_label'] == label].iterrows():
        folium.Marker(
            location=[row['lat_grid'], row['lon_grid']],
            icon=folium.Icon(color=color, icon='info-sign'),
            popup=(
                f"{label} Hotspot\n"
                f"Empty Rate: {row['empty_rate']*100:.2f}%\n"
                f"Total Pings: {row['total_pings']}\n"
                f"Empty Pings: {row['empty_pings']}"
            )
        ).add_to(cluster)

# Add layer control
folium.LayerControl().add_to(hotspot_map)

# Save the map
hotspot_map.save('empty_passenger_spot.html')
print("✅ Hotspot marker map saved as empty_passenger_spot.html")


✅ Hotspot marker map saved as empty_passenger_spot.html


In [23]:
# Hired Trip Hotspot Map Visualization
import folium
from folium.plugins import MarkerCluster

# Aggregate hired trip counts by grid and tourist label
hired_geo = df.groupBy('lat_grid', 'lon_grid', 'is_tourist_time').agg(
    F.sum(F.when(df.is_hired == 1, 1).otherwise(0)).alias('hired_trips'),
    F.count('*').alias('total_pings')
).toPandas()

# Add tourist label for color coding
hired_geo['tourist_label'] = hired_geo['is_tourist_time'].map({1: 'Tourist', 0: 'Non-Tourist'})

# Find top N hotspots by hired trips for each type
N = 10
hotspots = hired_geo.sort_values('hired_trips', ascending=False).groupby('tourist_label').head(N)

# Center map at mean coordinates
map_center = [hired_geo['lat_grid'].mean(), hired_geo['lon_grid'].mean()]
hired_map = folium.Map(location=map_center, zoom_start=12)

# Add marker clusters for each type
for label, color in zip(['Tourist', 'Non-Tourist'], ['green', 'purple']):
    cluster = MarkerCluster(name=f"{label} Hired Hotspots").add_to(hired_map)
    for _, row in hotspots[hotspots['tourist_label'] == label].iterrows():
        folium.Marker(
            location=[row['lat_grid'], row['lon_grid']],
            icon=folium.Icon(color=color, icon='taxi'),
            popup=(
                f"{label} Hired Hotspot\n"
                f"Hired Trips: {int(row['hired_trips'])}\n"
                f"Total Pings: {int(row['total_pings'])}"
            )
        ).add_to(cluster)

# Add layer control
folium.LayerControl().add_to(hired_map)

# Save the map
hired_map.save('hired_trip_hotspot_map.html')
print("✅ Hired trip hotspot marker map saved as hired_trip_hotspot_map.html")


✅ Hired trip hotspot marker map saved as hired_trip_hotspot_map.html


In [24]:
# Hired Trip Proximity Hotspot Map (Clustered by Proximity)
import folium
from folium.plugins import MarkerCluster

# Use the hired_geo DataFrame from previous cell
# Optionally, filter for only locations with significant hired trips
proximity_threshold = hired_geo['hired_trips'].quantile(0.90)  # Top 10% by hired trips
proximity_hotspots = hired_geo[hired_geo['hired_trips'] >= proximity_threshold]

# Create a new map centered at the mean coordinates
proximity_map = folium.Map(location=map_center, zoom_start=12)

# Add a single marker cluster for all high-proximity hired trip locations
proximity_cluster = MarkerCluster(name="Hired Trip Proximity Hotspots").add_to(proximity_map)

for _, row in proximity_hotspots.iterrows():
    folium.Marker(
        location=[row['lat_grid'], row['lon_grid']],
        icon=folium.Icon(color='orange', icon='taxi'),
        popup=(
            f"Hired Trip Proximity Hotspot\n"
            f"Hired Trips: {int(row['hired_trips'])}\n"
            f"Total Pings: {int(row['total_pings'])}\n"
            f"Type: {row['tourist_label']}"
        )
    ).add_to(proximity_cluster)

folium.LayerControl().add_to(proximity_map)
proximity_map.save('hired_trip_proximity_hotspot_map.html')
print("✅ Hired trip proximity hotspot map saved as hired_trip_proximity_hotspot_map.html")


✅ Hired trip proximity hotspot map saved as hired_trip_proximity_hotspot_map.html


In [25]:
# Hired Trip Hotspot Map: Tourist vs Non-Tourist Ratio
import folium
from folium.plugins import MarkerCluster

# Calculate ratio of hired trips between tourist and non-tourist times for each grid
pivot = hired_geo.pivot_table(
    index=['lat_grid', 'lon_grid'],
    columns='tourist_label',
    values='hired_trips',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Compute ratio (Tourist/Non-Tourist), handle division by zero
pivot['ratio'] = pivot['Tourist'] / (pivot['Non-Tourist'] + 1e-6)

# Find top N locations with highest ratio (tourist-dominated) and lowest ratio (non-tourist-dominated)
N = 10
top_tourist = pivot.sort_values('ratio', ascending=False).head(N)
top_nontourist = pivot.sort_values('ratio', ascending=True).head(N)

# Create a new map centered at the mean coordinates
ratio_map = folium.Map(location=map_center, zoom_start=12)

# Add markers for tourist-dominated hotspots
for _, row in top_tourist.iterrows():
    folium.Marker(
        location=[row['lat_grid'], row['lon_grid']],
        icon=folium.Icon(color='darkgreen', icon='star'),
        popup=(
            f"Tourist-Dominated Hotspot\n"
            f"Tourist Hired Trips: {int(row['Tourist'])}\n"
            f"Non-Tourist Hired Trips: {int(row['Non-Tourist'])}\n"
            f"Ratio: {row['ratio']:.2f}"
        )
    ).add_to(ratio_map)

# Add markers for non-tourist-dominated hotspots
for _, row in top_nontourist.iterrows():
    folium.Marker(
        location=[row['lat_grid'], row['lon_grid']],
        icon=folium.Icon(color='cadetblue', icon='star'),
        popup=(
            f"Non-Tourist-Dominated Hotspot\n"
            f"Tourist Hired Trips: {int(row['Tourist'])}\n"
            f"Non-Tourist Hired Trips: {int(row['Non-Tourist'])}\n"
            f"Ratio: {row['ratio']:.2f}"
        )
    ).add_to(ratio_map)

ratio_map.save('hired_trip_tourist_ratio_hotspot_map.html')
print("✅ Hired trip tourist ratio hotspot map saved as hired_trip_tourist_ratio_hotspot_map.html")


✅ Hired trip tourist ratio hotspot map saved as hired_trip_tourist_ratio_hotspot_map.html
